In [1]:
import pandas as pd

In [4]:
posts = pd.read_csv('./test/posts.csv', index_col=0)
comments = pd.read_csv('./test/comments.csv', index_col=0)
answers = pd.read_csv('./test/answers.csv', index_col=0)

In [23]:
posts.replace({pd.np.NaN: None}, inplace=True)
comments.replace({pd.np.NaN: None}, inplace=True)
answers.replace({pd.np.NaN: None}, inplace=True)
posts[posts['post_id'] == 6562]

,from_id,num_comments,owner_id,post_id,reply_owner_id,reply_post_id,text
0,31274240,2,31274240,6562,None,None,Забыл запостить бле!!!!


In [37]:
dialogs = []
#расмотри ситуацию когда пользователь ответил в комментариях кому-то
#тогда диалог будет строить так
#найдем комментарий где пользователь обратился к кому-то
#найдем комментарий к кому он обратился
#найдем последующую историю переписки

for user in posts['owner_id'].drop_duplicates():
    user_posts = posts[posts['owner_id'] == user]
    for post in user_posts['post_id'].drop_duplicates():
        dialog = ""
        post_comments = comments[comments['post_id']==post]
        post_answers = answers[answers['post_id']==post]
        
        anchor = post_answers[post_answers['from_id']==user & post_answers['reply_to_user'].bool()].head(1)
        
        if not anchor.empty:
            print(anchor)
            if post_comments[post_comments['comment_id'] == int(anchor['reply_to_comment'])]:
                first = post_comments[post_comments['comment_id'] == anchor['reply_to_comment']].head(1)
            elif post_answers[post_answers['comment_id'] == int(anchor['reply_to_comment'])]:
                first = post_answers[post_answers['comment_id'] == anchor['reply_to_comment']].head(1)
            else:
                continue

            dialog += first['text'] + '\n'
            dialog += anchor['text'] + '\n'

            prev = anchor
            for ans in post_answers.iterrows():
                if ans['reply_to_comment'] == prev['comment_id']:
                    dialog += ans['text'] + '\n'
                    prev = ans
            dialogs.append(dialog)

ValueError: bool cannot act on a non-boolean single element Series